<a href="https://colab.research.google.com/github/huyanhdo/phobert/blob/master/PhoBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import dependency

In [1]:
!git lfs install
!git clone https://huggingface.co/wonrax/phobert-base-vietnamese-sentiment

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
Cloning into 'phobert-base-vietnamese-sentiment'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 35 (delta 13), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (35/35), done.


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.2 MB/s 
     |████████████████████████████████| 86 kB 4.3 MB/s 
     |████████████████████████████████| 6.6 MB 26.0 MB/s 
     |████████████████████████████████| 596 kB 32.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Text Preprocessing

In [3]:
import numpy as np
import re

dict = [['hsd','hạn sử dụng'],["ship", "vận chuyển"],["shop", "cửa hàng"],["m", "mình"],["mik","mình"],["ko","không"],['sz','size'],['rv','review'],['lm','làm'],['vsinh','vệ sinh'],
["k","không"],["kh","không"],["khong","không"],["kg","không"],["khg","không"],["tl","trả lời"],['zời','vời'],['ạh','ạ'],['zay','vậy'],['kb','không biết'],
["rep","trả lời"],["r","rồi"],["fb","facebook"],["face","faceook"],["thanks","cảm ơn"],["thank","cảm ơn"],['zậy','vậy'],['hy','hi'],['hog','không'],
["tks","cảm ơn"],["tk","cảm ơn"],["ok","tốt"],["oki","tốt"],["okie","tốt"],["sp","sản phẩm"],['ms','mới'],['ktra','kiểm tra'],['lém','lắm'],['hông','không'],
["dc","được"],["vs","với"],["đt","điện thoại"],["thjk","thích"],["thik","thích"],["qá","quá"],['wa','quá'],['trl','trả lời'],['bâyh','bây giờ'],
["trể","trễ"],["bgjo","bao giờ"],["h","giờ"],["qa","quá"],["dep","đẹp"],["xau","xấu"],["ib","nhắn tin"],['j','gì'],['hjc','hic'],['ltinh','linh tinh'],
["cute","dễ thương"],["sz","size"],["good","tốt"],["god","tốt"],["bt","bình thường"],["mn", "mọi người"], ['cb','chuẩn bị'],['phèng','phèn'],
["mng", "mọi người"], ["cx", "cũng"], ["bth", "bình thường"], ["flashsale", "sale"], ["ncl", "nói chung"],['nc','nói chung'],['nchung','nói chung'],
["mjnk", "mình"], ["ths", "cảm ơn"], ["r", "rồi"], ["oce" , "tốt"] , ["nma", "nhưng mà"], ["j", "gì"],['nhưg','nhưng'],['nhma','nhưng mà'],['nmà','nhưng mà'],
["lm", "làm"] , ["trc", "trước"], ["nh", "nhưng"] ,["dk", "được"], ["ng", "người"], ["mk", "mình"], ["ak", "à"],['àk','à'],['nhmà','nhưng mà'],
["bthg", "bình thường"], ["đc", "được"],["vid", "video"],["vx", "vẫn"],['chx', 'chưa'],['+','với'],['nhìu','nhiều'],['ctay','chia tay'],['kh','không'],
['t','tôi'],['đb','đầu buồi'],['ntin','nhắn tin'],['onl','online'],['nt','nhắn tin'],['vãi chưởng','vc'],['ji','gì'],['vd','ví dụ'],['tl','trả lời'],['ae','anh em'],
['nnao','như nào'],['bửn','bẩn'],['okay','ok'],['lsao','làm sao'],['wa','quá'],['lg','lượng'],['tôt','tốt']]

def PreProcessing (text) :
    text1 = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), text, flags=re.IGNORECASE)
    text1 = re.sub(r'([),()])\1+', lambda m: m.group(1).upper(), text1, flags=re.IGNORECASE)
    text1 = text1.lower()
    text1 = re.sub("\n", " ", text1)
    # text1 = re.sub(r"([?.!,¿])", " ", text1)
    text1 = re.sub(r'[" "]+', " ", text1)
    
    list_text = text1.split(" ")
    remover = []
    for chars in list_text :
        if len(chars) > 7 :
            remover.append(chars)
    for chars in remover :
        list_text.remove(chars)
    for i in range(len(list_text)):
        for j in range(len(dict)) :
            if (list_text[i] == dict[j][0]):
                list_text[i] = dict[j][1]
    text2 = " ".join(list_text)
    return text2

Import Data and data preprocessing

In [4]:
from torch.utils.data import Dataset,DataLoader
class DataSet(Dataset):
  def __init__(self,data):
    self.data = data[1].values
    self.label = data[0].values
  
  def __getitem__(self,idx):
    label = torch.zeros(5)
    label[self.label[idx]-1] = 1 
    return PreProcessing(self.data[idx]),torch.tensor(label)
  
  def __len__(self):
    return len(self.data)

In [5]:
import pandas as pd 

In [6]:
crawl  = pd.read_csv('/content/data_clean.csv',encoding = 'utf8',header = None)
train  = pd.read_csv("/content/train_data.csv",encoding = 'utf8')
train = train.drop(columns=['Unnamed: 0'])
train.rename(columns = {'label':0,'sentence':1},inplace = True)
crawl.head(5)

,0,1
0,1,Đèn không sáng như nghĩ . Như đồ chơi của mấy bé
1,1,"Pin kém, mới dùng vài lần đã hỏng"
2,1,Quá tệ
3,1,Dùng nhanh hết pin quá chưa được 1h Sạc không ...
4,1,"Shop giao thiếu hàng, nhìn kiện hàng đã thấy t..."


In [7]:
one_star = train[train[0] == 1]
one_star = one_star.reset_index()
one_star.drop(columns = 'index',inplace = True)

two_star = train[train[0] == 2]
two_star = two_star.reset_index()
two_star.drop(columns = 'index',inplace = True)

three_star = train[train[0] == 3]
three_star = three_star.reset_index()
three_star.drop(columns = 'index',inplace = True)

four_star = train[train[0] == 4]
four_star = four_star.reset_index()
four_star.drop(columns = 'index',inplace = True)

five_star = train[train[0] == 5]
five_star = five_star.reset_index()
five_star.drop(columns = 'index',inplace = True)

crawl_data = crawl
crawl_data = crawl_data.reset_index()
crawl_data.drop(columns = 'index',inplace = True)
crawl_data.head(5)

,0,1
0,1,Đèn không sáng như nghĩ . Như đồ chơi của mấy bé
1,1,"Pin kém, mới dùng vài lần đã hỏng"
2,1,Quá tệ
3,1,Dùng nhanh hết pin quá chưa được 1h Sạc không ...
4,1,"Shop giao thiếu hàng, nhìn kiện hàng đã thấy t..."


In [8]:
from sklearn.model_selection import train_test_split
train_data = pd.DataFrame(columns = [0,1])
test_data = pd.DataFrame(columns = [0,1])

def split_data(data_set,train,test):
  x_train,x_test,y_train,y_test = train_test_split(data_set[1].values,data_set[0].values,test_size = 0.33)
  for x,y in zip(x_train,y_train):
    train = train.append({0:y,1:x},ignore_index = True)
  for x,y in zip(x_test,y_test):
    test = test.append({0:y,1:x},ignore_index = True)
  return train,test

train_data,test_data = split_data(one_star,train_data,test_data)
train_data,test_data = split_data(two_star,train_data,test_data)
train_data,test_data = split_data(three_star,train_data,test_data)
train_data,test_data = split_data(four_star,train_data,test_data)
train_data,test_data = split_data(five_star,train_data,test_data)
train_data,test_data = split_data(crawl_data,train_data,test_data)

In [9]:
print(len(train_data[train_data[0] == 1]))
print(len(train_data[train_data[0] == 2]))
print(len(train_data[train_data[0] == 3]))
print(len(train_data[train_data[0] == 4]))
print(len(train_data[train_data[0] == 5]))

1982
1938
2060
2173
1784


In [10]:
from sklearn.utils import shuffle

train_data = shuffle(train_data)
train_data.reset_index(inplace = True,drop = True)

test_data = shuffle(test_data)
test_data.reset_index(inplace = True,drop = True)

train_set = DataSet(train_data)
test_set = DataSet(test_data)

train_loader = DataLoader(train_set,batch_size = 32,shuffle = True)
test_loader =  DataLoader(test_set,batch_size = 32,shuffle = True)

print(len(train_set))
print(len(test_set))

9937
4900


In [ ]:
# a = next(iter(train_loader))
# # x = tokenizer(a[0],return_tensors='pt', padding=True, truncation=True).to(device)
# print(len(train_loader))
# print(len(train_set))
# # print(train_set[4])

Load pretrain model

In [11]:
import torch
from transformers import RobertaForSequenceClassification, AutoTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment")

tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False)

Downloading:   0%|          | 0.00/999 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/285 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [12]:
for i in model.parameters():
  i.requires_grad = False
model.classifier.out_proj = torch.nn.Linear(768,5)
model = model.to(device)

In [13]:
import torch.optim as optim
import matplotlib.pyplot as plt
optim = optim.Adam(model.parameters(),lr = 2e-5)
loss_func = torch.nn.CrossEntropyLoss()

In [ ]:
# sentence = torch.tensor([tokenizer.encode("haha")]).to(device)
# print(model(sentence))

In [14]:
from tqdm import tqdm
def train_model(model,loss_func,optimizer,train_loader):
  loss = []
  min = 1e9
  model.train()
  for epoch in range(3):
    loss_step = 0
    
    for batch,label in tqdm(train_loader):
      encoding= tokenizer(batch,return_tensors='pt', padding=True, truncation=True).to(device)
      input_ids = encoding['input_ids'].to(device)
   
      label = label.to(device)
      
      optim.zero_grad()

      predict = model(input_ids).logits.to(device)
    
      l = loss_func(predict,label).to(device)
      l.backward()
      optim.step()
      loss_step += l.item() * len(batch[0])
    
    loss_step = loss_step / 9937

    if min > loss_step:
      min = loss_step
      torch.save(model.state_dict(),'/content/best.pth')

    print('epoch: {}, loss {:4f} '.format(epoch,loss_step))
    loss.append(loss_step)

  plt.plot(loss)
  plt.show()

In [18]:
train_model(model,loss_func,optim,train_loader)

  0%|          | 0/311 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
  1%|          | 3/311 [00:19<33:00,  6.43s/it]


KeyboardInterrupt: ignored

In [15]:
def eval_model(model,test_loader):
  model.eval()
  cnt = 0
  df = pd.DataFrame(columns = ['label','prediction','text'])
  with torch.no_grad():
    for batch,label in test_loader:
      
      encoding= tokenizer(batch,return_tensors='pt', padding=True, truncation=True).to(device)
      input_ids = encoding['input_ids'].to(device)
      # attention_mask = encoding['attention_mask'].to(device)
      label = label.to(device)
      
      predict = model(input_ids).logits.to(device)
      # predict = model(input_ids,attention_mask = attention_mask).logits.to(device)

      _,pred = torch.max(predict.softmax(dim=-1),1)
      _,label = torch.max(label,1)
      cnt += (pred == label).sum().item()
      for idx in range(len(pred)):
        if(pred[idx]!=label[idx]):
          df = df.append({'label':label[idx].item(),'prediction':pred[idx].item(),'text':batch[idx]} ,ignore_index = True)
           # print('label '+str(label[idx].item())+' predict '+str(pred[idx].item())+' sentence '+batch[idx])
    print('accuracy:{}'.format(100*cnt/4900))
    df.to_csv('/content/res.csv',index = None)

In [ ]:
# model.load_state_dict(torch.load('/content/best.pth'))
eval_model(model,test_loader)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


accuracy:19.836734693877553


In [ ]:
model.eval()
predict = model(torch.tensor([tokenizer.encode("Đẹp vãi cả chưởng")]).to(device))
_,x = torch.max(predict.logits.softmax(dim = -1),1)

print(predict.logits)
print(x+1)

predict = model(torch.tensor([tokenizer.encode("quá tệ")]).to(device))
_,x = torch.max(predict.logits.softmax(dim = -1),1)

print(predict.logits)
print(x+1)

In [ ]:
print(model)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN